In [57]:
# Install all required packages
!pip install google-adk nltk google-generativeai
print("✅ All libraries installed successfully!")

✅ All libraries installed successfully!


In [58]:
import os
from kaggle_secrets import UserSecretsClient
import nltk
from nltk.tokenize import sent_tokenize
from google.adk.tools import AgentTool, ToolContext, load_memory
from google.adk.models.google_llm import Gemini
from google.genai import types
from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [59]:
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [86]:
# Gets the proxied URL in the Kaggle Notebooks environment
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [87]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504]  # Retry on these HTTP errors
)


async def auto_save_to_memory(callback_context):
    """Automatically persists session data to long-term memory."""
    if hasattr(callback_context._invocation_context, 'memory_service'):
        await callback_context._invocation_context.memory_service.add_session_to_memory(
            callback_context._invocation_context.session
        )


# Tool to save user preferences into Session State
def set_user_context(tool_context: ToolContext, role: str, focus_area: str):
    """
    Saves the user's role and focus area to the current session state.
    Use this when the user introduces themselves (e.g., "I am a Product Manager").
    """
    tool_context.state["user:role"] = role
    tool_context.state["user:focus"] = focus_area
    return f"Context saved: Role={role}, Focus={focus_area}"


print("✅ Retry config and callbacks defined.")

✅ Retry config and callbacks defined.


In [88]:
# --- FALLBACK TOOLS ---
def offline_meeting_fallback(text: str):
    sentences = sent_tokenize(text)
    actions = [s for s in sentences if any(w in s.lower() for w in ['todo', 'action', 'deadline'])]
    return f"[OFFLINE FALLBACK] Actions detected: {len(actions)}"


def offline_research_fallback(text: str):
    sentences = sent_tokenize(text)
    tldr = sentences[0] if sentences else "No text"
    return f"[OFFLINE FALLBACK] TL;DR: {tldr}"


print("✅ Fallback tools defined.")

✅ Fallback tools defined.


In [89]:
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

print("✅ Model initialized.")

✅ Model initialized.


In [90]:
# [A] Meeting Agent
meeting_agent = Agent(
    name="MeetingPrepAgent",
    model=model,
    instruction="Analyze the transcript. Output: Pre-Brief, Agenda, and Action Items.",
    tools=[offline_meeting_fallback]
)

print("✅ Meeting Agent defined.")

✅ Meeting Agent defined.


In [91]:
# [B] Research Pipeline
extractor = Agent(
    name="FactExtractor",
    model=model,
    instruction="Extract Top 5 Key Findings and Sources from the provided content.",
    output_key="facts"
)

writer = Agent(
    name="BriefWriter",
    model=model,
    instruction="Using the previous agent's extracted findings, write a Decision-Ready Brief with TL;DR, Findings, and Next Steps.",
)

research_pipeline = SequentialAgent(
    name="ResearchSummaryAgent",
    sub_agents=[extractor, writer]
)

print("✅ Research Pipeline defined.")

✅ Research Pipeline defined.


In [92]:
# [C] Orchestrator (Router with Memory)
root_agent = Agent(
    name="AgentAssist_Orchestrator",
    model=model,
    instruction="""You are the AgentAssist Orchestrator.
    
    1. CHECK CONTEXT: Use 'load_memory' to see if you know the user's preferences.
    2. SAVE CONTEXT: If the user states their role/job, use 'set_user_context'.
    3. ROUTE:
       - Meeting notes -> 'MeetingPrepAgent'
       - Documents -> 'ResearchSummaryAgent'
       - General -> Answer directly using Google Search if needed
    """,
    tools=[AgentTool(meeting_agent), AgentTool(research_pipeline), google_search, load_memory, set_user_context],
    after_agent_callback=auto_save_to_memory  # Enables auto-memory storage
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [93]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)
print("✅ Root Agent defined.")

✅ Root Agent defined.


In [94]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [95]:
# Test query
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
helpful_assistant > The Agent Development Kit (ADK) from Google is an open-source, flexible, and modular framework designed to simplify the development and deployment of AI agents and multi-agent systems. It aims to make agent development more akin to traditional software development by providing a structured way to build, orchestrate, and evaluate agents powered by large language models (LLMs).

Key aspects of the ADK include:
*   **Purpose:** It allows developers to define agent behavior, orchestration, and tool usage directly in code, offering fine-grained control and enabling robust debugging, versioning, and deployment across various environments, from local machines to the cloud.
*   **Flexibility:** While optimized for Gemini and the Google ecosystem, ADK is model-agnostic and deployment-agnostic, meaning it can work with different LLMs (such as 

In [ ]:
!adk create Ai-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [103]:
url_prefix = get_adk_proxy_url()

In [104]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [215]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: